# FLAT input data prepping

This code prepares the final input file to the FLAT model. 

**Original code:** [Konstantinos Pegios](https://github.com/kopegios) <br />
**Conceptualization & Methodological review :** [Alexandros Korkovelos](https://github.com/akorkovelos) & [Konstantinos Pegios](https://github.com/kopegios)<br />
**Updates, Modifications:** [Alexandros Korkovelos](https://github.com/akorkovelos)<br />
**Funding:** The World Bank (contract number: 7190531), [KTH](https://www.kth.se/en/itm/inst/energiteknik/forskning/desa/welcome-to-the-unit-of-energy-systems-analysis-kth-desa-1.197296)

In [1]:
#Import modules and libraries

import geopandas as gpd
import rasterio as rio
import pandas as pd
import fiona
import gdal
import osr
import ogr
import rasterio.mask
import time
import os
import ogr, gdal, osr, os
import numpy as np
import itertools
import re

from osgeo import gdal,ogr
import struct
import csv

import glob

from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling
from rasterstats import point_query, zonal_stats
from pyproj import Proj
from shapely.geometry import Point, Polygon

In [2]:
# Import data 

#path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data"
name_of_file = "Ben_Maize_Harv_2000_admin2_1km.csv"

In [3]:
# Import csv as pandas dataframe
df = pd.read_csv(path + "\\" + name_of_file)

In [4]:
#df = df.drop(["yield", "production_ha"], axis=1)
df.rename(columns={'id': 'alloc_key'}, inplace=True)

In [5]:
df.rename(columns={'COUNTRY_COUNTRY': 'state'}, inplace=True)
df.rename(columns={'X_deg': 'lon'}, inplace=True)
df.rename(columns={'Y_deg': 'lat'}, inplace=True)
df.rename(columns={'area_ha': 'statearea_ha'}, inplace=True)
df.rename(columns={'country_code': 'c_code'}, inplace=True)

In [6]:
df.head()

,state,c_code,country,crop,year,harea_2000,yield,prduction_ha,shareofsum,harv_area_ha,district_a,shareofdis,statearea_ha,perimeter_km,lon,lat,alloc_key
0,Cobli,BEN,Benin,Maize,2000,2970,0.794877,4346,0.004722,5467.509659,93656.60451,0.058378,93656.58706,12.752486,0.782724,10.504024,1
1,Cobli,BEN,Benin,Maize,2000,2970,0.794877,4346,0.004722,5467.509659,93656.60451,0.058378,93656.58706,12.752486,0.782547,10.498548,2
2,Cobli,BEN,Benin,Maize,2000,2970,0.794877,4346,0.004722,5467.509659,93656.60451,0.058378,93656.58706,12.752486,0.782843,10.487902,3
3,Cobli,BEN,Benin,Maize,2000,2970,0.794877,4346,0.004722,5467.509659,93656.60451,0.058378,93656.58706,12.752486,0.782522,10.479100,4
4,Cobli,BEN,Benin,Maize,2000,2970,0.794877,4346,0.004722,5467.509659,93656.60451,0.058378,93656.58706,12.752486,0.782125,10.470444,5


In [7]:
#Function to change the order of columns in the dataframe 
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

In [8]:
# Allocate order in dataframe columns
df = change_column_order(df, 'alloc_key', 1)
df = change_column_order(df, 'lon', 2)
df = change_column_order(df, 'lat', 3)
df = change_column_order(df, 'country', 4)
df = change_column_order(df, 'c_code', 5)
df = change_column_order(df, 'state', 6)
df = change_column_order(df, 'statearea_ha', 7)
df = change_column_order(df, 'perimeter_km', 8)
df = change_column_order(df, 'year', 9)
df = change_column_order(df, 'crop', 10)
df = change_column_order(df, 'harv_area_ha', 11)

In [9]:
df.head()

,alloc_key,lon,lat,country,c_code,state,statearea_ha,perimeter_km,year,harea_2000,crop,harv_area_ha,yield,prduction_ha,shareofsum,district_a,shareofdis
0,1,0.782724,10.504024,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378
1,2,0.782547,10.498548,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378
2,3,0.782843,10.487902,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378
3,4,0.782522,10.479100,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378
4,5,0.782125,10.470444,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378


In [10]:
# Add geometry and convert to spatial dataframe in source CRS
df['geometry'] = list(zip(df['lon'], df['lat']))
df['geometry'] = df['geometry'].apply(Point)
df = gpd.GeoDataFrame(df, geometry='geometry', crs={'init': 'epsg:4326'})

# Reproject data in to Ordnance Survey GB coordinates
df_osgb = df.to_crs({'init': 'epsg:4326'})

C:\Users\oluchi\.conda\envs\agrodem_run\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [11]:
df_osgb.head()

,alloc_key,lon,lat,country,c_code,state,statearea_ha,perimeter_km,year,harea_2000,crop,harv_area_ha,yield,prduction_ha,shareofsum,district_a,shareofdis,geometry
0,1,0.782724,10.504024,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378,POINT (0.78272 10.50402)
1,2,0.782547,10.498548,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378,POINT (0.78255 10.49855)
2,3,0.782843,10.487902,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378,POINT (0.78284 10.48790)
3,4,0.782522,10.479100,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378,POINT (0.78252 10.47910)
4,5,0.782125,10.470444,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378,POINT (0.78212 10.47044)


<div class="alert alert-block alert-warning">
<b>Note:</b> The geodataframe, contains crop points with needed attributes including a geometry column. It needs to be exported as shapefile. However, when this is done with geopandas, the column geometry (Point(x,y)) is not transfered in the attribute table. This is however needed for the extraction process below, otherwise "None" values are returned. In order to overcome this issue, the csv is imported into Qgis and then exported as feature layer in the same folder. This solves the issue. 
</div>

In [12]:
# Define output path
# Overwriting the csv file
#path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data"
shpname = "Ben_Maize_Harv_2000_admin2_1km_new.csv"

#MAIZE
df_osgb.to_csv(os.path.join(path,"{c}".format(c=shpname)), index=False)
df_osgb.to_file('Ben_Maize_Harv_2000_admin2_1km_new.shp',driver = 'ESRI Shapefile')
# Define output path
#path = r"N:\Agrodem\Downscaling\Output_Data\Crop_Maps"
#path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Crop_Maps"

#Maize


#### >>> Please open Qgis and act as suggested above<<<

In [13]:
y = gpd.read_file(r'C:\Github\agrodem\agrodem_preprocessing\Downscaling\Downscaling_prepping\Ben_Maize_Harv_2000_admin2_1km_new.shp')

In [14]:
y.head()

,alloc_key,lon,lat,country,c_code,state,statearea_,perimeter_,year,harea_2000,crop,harv_area_,yield,prduction_,shareofsum,district_a,shareofdis,geometry
0,1,0.782724,10.504024,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378,POINT (0.78272 10.50402)
1,2,0.782547,10.498548,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378,POINT (0.78255 10.49855)
2,3,0.782843,10.487902,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378,POINT (0.78284 10.48790)
3,4,0.782522,10.479100,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378,POINT (0.78252 10.47910)
4,5,0.782125,10.470444,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,Maize,5467.509659,0.794877,4346,0.004722,93656.60451,0.058378,POINT (0.78212 10.47044)


## Extracting raster input to csv 

In [15]:
#raster_path = r"N:\Agrodem\Downscaling\Input_Data\rasters"
raster_path = r"C:\Oluchi\Downscaling\Input_Data\raster_new"
raster_files =[]
#csvoutpath = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input\1km_Rice"
csvoutpath = r"C:\Oluchi\Downscaling\Output_Data\FLAT_Input\1km_Maize"

print ("Reading independent variables...")

for i in os.listdir(raster_path):
    if i.endswith('.tif'):
        raster_files.append(i)  

#shp_filename = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops\shapefiles\Moz_Rice_Harv_2000_admin2_1km.shp"
shp_filename = r'C:\Github\agrodem\agrodem_preprocessing\Downscaling\Downscaling_prepping\Ben_Maize_Harv_2000_admin2_1km_new.shp'
print ("Extracting raster values to points...")

for i in raster_files:
    print("Extracting " + i + " values...")
    src_filename = raster_path + "\\" + i 
    li_values = list()

    src_ds=gdal.Open(src_filename) 
    gt=src_ds.GetGeoTransform()
    rb=src_ds.GetRasterBand(1)
    ds=ogr.Open(shp_filename)    
    lyr=ds.GetLayer()
    
    for feat in lyr:
        geom = feat.GetGeometryRef()
        feat_id = feat.GetField('alloc_key')
        mx,my=geom.GetX(), geom.GetY()  #coord in map units

        #Convert from map to pixel coordinates.
        #Only works for geotransforms with no rotation.
        px = int((mx - gt[0]) / gt[1]) #x pixel
        py = int((my - gt[3]) / gt[5]) #y pixel

        intval=rb.ReadAsArray(px,py,1,1)
        li_values.append([feat_id, intval[0]])
        
    print ("Writing " + i + " values to csv...")
    
    with open(csvoutpath + "\\" + i.split('.')[0] + '.csv', 'w') as csvfile:
        wr = csv.writer(csvfile)
        wr.writerows(li_values)

Reading independent variables...
Extracting raster values to points...
Extracting BDRICM.tif values...
Writing BDRICM.tif values to csv...
Extracting BLD.tif values...
Writing BLD.tif values to csv...
Extracting CLYPPT.tif values...
Writing CLYPPT.tif values to csv...
Extracting DRAINFAO.tif values...
Writing DRAINFAO.tif values to csv...
Extracting GFSAD-landcover.tif values...
Writing GFSAD-landcover.tif values to csv...
Extracting landcover.tif values...
Writing landcover.tif values to csv...
Extracting MODIS-Band2.tif values...
Writing MODIS-Band2.tif values to csv...
Extracting modis-evi.tif values...
Writing modis-evi.tif values to csv...
Extracting modis-ndvi.tif values...
Writing modis-ndvi.tif values to csv...
Extracting ORCDRC.tif values...
Writing ORCDRC.tif values to csv...
Extracting PHIHOX.tif values...
Writing PHIHOX.tif values to csv...
Extracting prec.tif values...
Writing prec.tif values to csv...
Extracting srad.tif values...
Writing srad.tif values to csv...
Extract

## Merge csv files with crop

In [81]:
#Import data 

#path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data"
name_of_file = "Ben_Maize_Harv_2000_admin2_1km_new.csv"

flat_input = pd.read_csv(path + "\\" + name_of_file)

In [82]:
#csvoutpath = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input\1km_Rice"
csvoutpath = r"C:\Oluchi\Downscaling\Output_Data\FLAT_Input\1km_Maize"
csv_files = []

print ("Reading csv files...")

for i in os.listdir(csvoutpath):
    if i.endswith('.csv'):
        csv_files.append(i)  

for i in csv_files:
    print('Reading...'+ i)    
    df_csv = pd.read_csv(csvoutpath + "//" + i, index_col=None, header=None)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].astype(str)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace('[','')
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace(']','')
    columnName = i.split('.')[0]
    print("Merging..." + columnName)
    flat_input[columnName] = df_csv.iloc[:,1]

Reading csv files...
Reading...BDRICM.csv
Merging...BDRICM
Reading...BLD.csv
Merging...BLD
Reading...CLYPPT.csv
Merging...CLYPPT
Reading...DRAINFAO.csv
Merging...DRAINFAO
Reading...GFSAD-landcover.csv
Merging...GFSAD-landcover
Reading...landcover.csv
Merging...landcover
Reading...MODIS-Band2.csv
Merging...MODIS-Band2
Reading...modis-evi.csv
Merging...modis-evi
Reading...modis-ndvi.csv
Merging...modis-ndvi
Reading...ORCDRC.csv
Merging...ORCDRC
Reading...PHIHOX.csv
Merging...PHIHOX
Reading...prec.csv
Merging...prec
Reading...srad.csv
Merging...srad
Reading...SRTM-elevation.csv
Merging...SRTM-elevation
Reading...tavg.csv
Merging...tavg
Reading...TEXMHT.csv
Merging...TEXMHT
Reading...wind.csv
Merging...wind


## Cleaning and normalizing dataframe

In [83]:
#Show columns
flat_input.columns

Index(['alloc_key', 'lon', 'lat', 'country', 'c_code', 'state', 'statearea_ha',
       'perimeter_km', 'year', 'harea_2000', 'crop', 'harv_area_ha', 'yield',
       'prduction_ha', 'shareofsum', 'district_a', 'shareofdis', 'geometry',
       'BDRICM', 'BLD', 'CLYPPT', 'DRAINFAO', 'GFSAD-landcover', 'landcover',
       'MODIS-Band2', 'modis-evi', 'modis-ndvi', 'ORCDRC', 'PHIHOX', 'prec',
       'srad', 'SRTM-elevation', 'tavg', 'TEXMHT', 'wind'],
      dtype='object')

In [84]:

flat_input.head(10)

,alloc_key,lon,lat,country,c_code,state,statearea_ha,perimeter_km,year,harea_2000,...,modis-evi,modis-ndvi,ORCDRC,PHIHOX,prec,srad,SRTM-elevation,tavg,TEXMHT,wind
0,1,0.782724,10.504024,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,2409.,4483.,8.,64.,89.333336,18743.166,167,28.499998,6.,2.2
1,2,0.782547,10.498548,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,2564.,4024.,8.,64.,89.333336,18739.,168,28.508331,6.,2.2
2,3,0.782843,10.487902,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,2466.,3670.,8.,64.,89.166664,18719.584,159,28.550001,9.,2.2
3,4,0.782522,10.479100,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,2165.,3494.,8.,64.,89.25,18698.834,163,28.499998,9.,2.1
4,5,0.782125,10.470444,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,2110.,3291.,8.,64.,89.416664,18693.,158,28.491667,6.,2.
5,6,0.781715,10.461530,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,2761.,4412.,9.,64.,89.166664,18672.75,152,28.483332,6.,2.
6,7,0.781303,10.452552,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,2542.,3968.,7.,64.,89.,18666.416,144,28.474998,6.,2.1
7,8,0.780885,10.443544,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,2630.,4826.,9.,64.,89.,18672.25,146,28.474998,6.,2.1
8,9,0.780420,10.434519,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,2439.,3945.,7.,64.,89.083336,18667.75,151,28.433332,9.,2.1
9,10,0.779943,10.425500,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,2547.,3980.,8.,64.,89.333336,18664.084,159,28.40833,9.,2.1


In [85]:
#Delete geometry column
flat_input = flat_input.drop(["geometry"], axis=1)

#### Convert values in newly added columns into float 

In [86]:
flat_input["harv_area_ha"] = flat_input["harv_area_ha"].astype(float)
flat_input["BDRICM"] = flat_input["BDRICM"].astype(float)
flat_input["BLD"] = flat_input["BLD"].astype(float)
flat_input["CLYPPT"] = flat_input["CLYPPT"].astype(float)
flat_input["DRAINFAO"] = flat_input["DRAINFAO"].astype(float)
flat_input["GFSAD-landcover"] = flat_input["GFSAD-landcover"].astype(float)
flat_input["landcover"] = flat_input["landcover"].astype(float)
flat_input["MODIS-Band2"] = flat_input["MODIS-Band2"].astype(float)
flat_input["modis-evi"] = flat_input["modis-evi"].astype(float)
flat_input["modis-ndvi"] = flat_input["modis-ndvi"].astype(float)
flat_input["ORCDRC"] = flat_input["ORCDRC"].astype(float)
flat_input["PHIHOX"] = flat_input["PHIHOX"].astype(float)
flat_input["prec"] = flat_input["prec"].astype(float)
flat_input["srad"] = flat_input["srad"].astype(float)
flat_input["SRTM-elevation"] = flat_input["SRTM-elevation"].astype(float)
flat_input["tavg"] = flat_input["tavg"].astype(float)
flat_input["TEXMHT"] = flat_input["TEXMHT"].astype(float)
flat_input["wind"] = flat_input["wind"].astype(float)

In [87]:
flat_input.dtypes

alloc_key            int64
lon                float64
lat                float64
country             object
c_code              object
state               object
statearea_ha       float64
perimeter_km       float64
year                 int64
harea_2000           int64
crop                object
harv_area_ha       float64
yield              float64
prduction_ha         int64
shareofsum         float64
district_a         float64
shareofdis         float64
BDRICM             float64
BLD                float64
CLYPPT             float64
DRAINFAO           float64
GFSAD-landcover    float64
landcover          float64
MODIS-Band2        float64
modis-evi          float64
modis-ndvi         float64
ORCDRC             float64
PHIHOX             float64
prec               float64
srad               float64
SRTM-elevation     float64
tavg               float64
TEXMHT             float64
wind               float64
dtype: object

#### Normalize EVI and NDVI 

In [88]:
flat_input['modis-evi'] = flat_input['modis-evi']*0.0001
flat_input['modis-ndvi'] = flat_input['modis-ndvi']*0.0001

In [89]:
flat_input.head()

,alloc_key,lon,lat,country,c_code,state,statearea_ha,perimeter_km,year,harea_2000,...,modis-evi,modis-ndvi,ORCDRC,PHIHOX,prec,srad,SRTM-elevation,tavg,TEXMHT,wind
0,1,0.782724,10.504024,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,0.2409,0.4483,8.0,64.0,89.333336,18743.166,167.0,28.499998,6.0,2.2
1,2,0.782547,10.498548,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,0.2564,0.4024,8.0,64.0,89.333336,18739.000,168.0,28.508331,6.0,2.2
2,3,0.782843,10.487902,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,0.2466,0.3670,8.0,64.0,89.166664,18719.584,159.0,28.550001,9.0,2.2
3,4,0.782522,10.479100,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,0.2165,0.3494,8.0,64.0,89.250000,18698.834,163.0,28.499998,9.0,2.1
4,5,0.782125,10.470444,Benin,BEN,Cobli,93656.58706,12.752486,2000,2970,...,0.2110,0.3291,8.0,64.0,89.416664,18693.000,158.0,28.491667,6.0,2.0


#### Drop  areas that are indicated as non-cropland in Global Food Security-support Analysis Data at 30m(GFSAD30) [here](https://developers.google.com/earth-engine/datasets/catalog/USGS_GFSAD1000_V0#bands)

In [90]:
flat_input.drop(flat_input[flat_input["GFSAD-landcover"] == 0].index, inplace=True)
#flat_input = flat_input[flat_input["Landcover"] == ]
#flat_input = flat_input.drop(["Landcover"], axis=1)
flat_input["GFSAD-landcover"]

1         8.0
2         8.0
3         8.0
4         8.0
5         8.0
         ... 
105553    8.0
105554    8.0
105557    8.0
105558    8.0
105559    8.0
Name: GFSAD-landcover, Length: 49394, dtype: float64

#### Fixing out-of-range values

In [91]:
# remove blank spaces in state names    
flat_input["state"].replace('\s+', '_',regex=True,inplace=True)
flat_input["state"].replace("'", '_',regex=True,inplace=True)

In [92]:
# Re-indexing allocation keys to avoid dublicates
flat_input = flat_input.assign(alloc_key=np.arange(len(flat_input))).reset_index(drop=True)

In [93]:
# Turning NaN rows to 0
flat_input.fillna(0,inplace=True)

In [94]:
# Turn all negative values to 0
pred_columns = ['tavg', 'srad', 'prec', 'wind', 'PHIHOX',
                'BDRICM', 'BLD', 'CLYPPT', 'TEXMHT', 'ORCDRC',
                'DRAINFAO', 'MODIS-Band2', 'SRTM-elevation']

for col in pred_columns:
    mvalue = flat_input[col].mean()
    print (mvalue)
    flat_input[col][flat_input[col]<0] = 0

-inf
17861.112347916755
78.8812777540592
-4.818398995829453e+34
62.34346276875734
141.126634813945
1371.9659067902985
18.956249746932826
8.19305988581609
4.114042191359275
3.816900838158481
3379.0889818653877
225.62424585982103


C:\Users\oluchi\.conda\envs\agrodem_run\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [95]:
for col in pred_columns:
    mvalue = flat_input[col].mean()
    print (mvalue)

27.53076778760578
17861.112347916755
83.5250806450986
2.2321334575049603
62.34346276875734
141.126634813945
1380.8729805239502
18.956249746932826
8.19305988581609
13.021115925011134
3.816900838158481
3379.0889818653877
225.62432684131676


In [96]:
flat_input.columns

Index(['alloc_key', 'lon', 'lat', 'country', 'c_code', 'state', 'statearea_ha',
       'perimeter_km', 'year', 'harea_2000', 'crop', 'harv_area_ha', 'yield',
       'prduction_ha', 'shareofsum', 'district_a', 'shareofdis', 'BDRICM',
       'BLD', 'CLYPPT', 'DRAINFAO', 'GFSAD-landcover', 'landcover',
       'MODIS-Band2', 'modis-evi', 'modis-ndvi', 'ORCDRC', 'PHIHOX', 'prec',
       'srad', 'SRTM-elevation', 'tavg', 'TEXMHT', 'wind'],
      dtype='object')

In [97]:
flat_input = change_column_order(flat_input, 'alloc_key', 0)
flat_input = change_column_order(flat_input, 'state', 1)
flat_input = change_column_order(flat_input, 'lon', 2)
flat_input = change_column_order(flat_input, 'lat', 3)
flat_input = change_column_order(flat_input, 'country', 4)
flat_input = change_column_order(flat_input, 'c_code', 5)

flat_input = change_column_order(flat_input, 'statearea_ha', 6)
flat_input = change_column_order(flat_input, 'perimeter_km', 7)
flat_input = change_column_order(flat_input, 'year', 8)
flat_input = change_column_order(flat_input, 'crop', 9)
flat_input = change_column_order(flat_input, 'harv_area_ha', 10)
flat_input = change_column_order(flat_input, 'yield', 11)
flat_input = change_column_order(flat_input, 'prduction_ha', 12)

flat_input = change_column_order(flat_input, 'tavg', 13)
flat_input = change_column_order(flat_input, 'srad', 14)
flat_input = change_column_order(flat_input, 'prec', 15)
flat_input = change_column_order(flat_input, 'wind', 16)

flat_input = change_column_order(flat_input, 'PHIHOX', 17)
flat_input = change_column_order(flat_input, 'BDRICM', 18)
flat_input = change_column_order(flat_input, 'BLD', 19)
flat_input = change_column_order(flat_input, 'CLYPPT', 20)
flat_input = change_column_order(flat_input, 'TEXMHT', 21)
flat_input = change_column_order(flat_input, 'ORCDRC', 22)
flat_input = change_column_order(flat_input, 'DRAINFAO', 23)

flat_input = change_column_order(flat_input, 'modis-evi', 24)
flat_input = change_column_order(flat_input, 'modis-ndvi', 25)

flat_input = change_column_order(flat_input, 'MODIS-Band2', 26)
flat_input = change_column_order(flat_input, 'SRTM-elevation', 27)
flat_input = change_column_order(flat_input, 'landcover', 28)
flat_input = change_column_order(flat_input, 'GFSAD-landcover', 28)

In [98]:
flat_input.head()

,alloc_key,state,lon,lat,country,c_code,statearea_ha,perimeter_km,year,crop,...,modis-evi,modis-ndvi,MODIS-Band2,SRTM-elevation,GFSAD-landcover,landcover,harea_2000,shareofsum,district_a,shareofdis
0,0,Cobli,0.782547,10.498548,Benin,BEN,93656.58706,12.752486,2000,Maize,...,0.2564,0.4024,3063.500000,168.0,8.0,9.0,2970,0.004722,93656.60451,0.058378
1,1,Cobli,0.782843,10.487902,Benin,BEN,93656.58706,12.752486,2000,Maize,...,0.2466,0.3670,3043.692308,159.0,8.0,14.0,2970,0.004722,93656.60451,0.058378
2,2,Cobli,0.782522,10.479100,Benin,BEN,93656.58706,12.752486,2000,Maize,...,0.2165,0.3494,3121.000000,163.0,8.0,14.0,2970,0.004722,93656.60451,0.058378
3,3,Cobli,0.782125,10.470444,Benin,BEN,93656.58706,12.752486,2000,Maize,...,0.2110,0.3291,3104.714286,158.0,8.0,9.0,2970,0.004722,93656.60451,0.058378
4,4,Cobli,0.781715,10.461530,Benin,BEN,93656.58706,12.752486,2000,Maize,...,0.2761,0.4412,3121.000000,152.0,8.0,14.0,2970,0.004722,93656.60451,0.058378


In [99]:
flat_input.columns

Index(['alloc_key', 'state', 'lon', 'lat', 'country', 'c_code', 'statearea_ha',
       'perimeter_km', 'year', 'crop', 'harv_area_ha', 'yield', 'prduction_ha',
       'tavg', 'srad', 'prec', 'wind', 'PHIHOX', 'BDRICM', 'BLD', 'CLYPPT',
       'TEXMHT', 'ORCDRC', 'DRAINFAO', 'modis-evi', 'modis-ndvi',
       'MODIS-Band2', 'SRTM-elevation', 'GFSAD-landcover', 'landcover',
       'harea_2000', 'shareofsum', 'district_a', 'shareofdis'],
      dtype='object')

#### Final column fixing

In [100]:
# give crop name
crop_modelled ="Maize"

# Dropping columns
flat_input = flat_input.drop(["country"], axis=1)
flat_input = flat_input.drop(["c_code"], axis=1)
flat_input = flat_input.drop(["perimeter_km"], axis=1)
flat_input = flat_input.drop(["year"], axis=1)
flat_input = flat_input.drop(["crop"], axis=1)
flat_input = flat_input.drop(["yield"], axis=1)
flat_input = flat_input.drop(["prduction_ha"], axis=1)
flat_input = flat_input.drop(["landcover"], axis=1)
flat_input = flat_input.drop(["harea_2000"], axis=1)
flat_input = flat_input.drop(["shareofsum"], axis=1)  
flat_input = flat_input.drop(["district_a"], axis=1)  
flat_input = flat_input.drop(["shareofdis"], axis=1)

#Remaning columns
flat_input.rename(columns={'state': 'NAME'}, inplace=True)
flat_input.rename(columns={'statearea_ha': 'statearea'}, inplace=True)
flat_input.rename(columns={'harv_area_ha': crop_modelled}, inplace=True)


In [101]:
flat_input

,alloc_key,NAME,lon,lat,statearea,Maize,tavg,srad,prec,wind,...,BLD,CLYPPT,TEXMHT,ORCDRC,DRAINFAO,modis-evi,modis-ndvi,MODIS-Band2,SRTM-elevation,GFSAD-landcover
0,0,Cobli,0.782547,10.498548,93656.58706,5467.509659,28.508331,18739.000,89.333336,2.2,...,1560.0,24.0,6.0,8.0,3.0,0.2564,0.4024,3063.500000,168.0,8.0
1,1,Cobli,0.782843,10.487902,93656.58706,5467.509659,28.550001,18719.584,89.166664,2.2,...,1480.0,19.0,9.0,8.0,3.0,0.2466,0.3670,3043.692308,159.0,8.0
2,2,Cobli,0.782522,10.479100,93656.58706,5467.509659,28.499998,18698.834,89.250000,2.1,...,1480.0,19.0,9.0,8.0,3.0,0.2165,0.3494,3121.000000,163.0,8.0
3,3,Cobli,0.782125,10.470444,93656.58706,5467.509659,28.491667,18693.000,89.416664,2.0,...,1500.0,23.0,6.0,8.0,3.0,0.2110,0.3291,3104.714286,158.0,8.0
4,4,Cobli,0.781715,10.461530,93656.58706,5467.509659,28.483332,18672.750,89.166664,2.0,...,1480.0,21.0,6.0,9.0,3.0,0.2761,0.4412,3121.000000,152.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49389,49389,Segbana,3.839690,10.658308,435828.86050,12578.954040,27.099998,17840.500,87.666664,2.6,...,1240.0,14.0,9.0,10.0,4.0,0.2980,0.5074,3042.555556,343.0,8.0
49390,49390,Segbana,3.839550,10.651491,435828.86050,12578.954040,27.099998,17840.500,87.666664,2.6,...,1180.0,11.0,9.0,12.0,4.0,0.2694,0.4504,3036.235294,346.0,8.0
49391,49391,Segbana,3.845997,10.666531,435828.86050,12578.954040,27.125000,17842.666,87.500000,2.6,...,1210.0,14.0,9.0,11.0,4.0,0.2560,0.4397,3042.083333,330.0,8.0
49392,49392,Segbana,3.846404,10.658197,435828.86050,12578.954040,27.058334,17830.666,87.750000,2.6,...,1240.0,14.0,9.0,10.0,4.0,0.2618,0.4879,3043.700000,341.0,8.0


In [102]:
flat_input.columns

Index(['alloc_key', 'NAME', 'lon', 'lat', 'statearea', 'Maize', 'tavg', 'srad',
       'prec', 'wind', 'PHIHOX', 'BDRICM', 'BLD', 'CLYPPT', 'TEXMHT', 'ORCDRC',
       'DRAINFAO', 'modis-evi', 'modis-ndvi', 'MODIS-Band2', 'SRTM-elevation',
       'GFSAD-landcover'],
      dtype='object')

In [103]:
# This part prints full results

#path = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input"
path =r"C:\Oluchi\Downscaling\Output_Data\FLAT_Input"
name_of_flat_input_file = "flat_input_Maize_1km"

flat_input.to_csv(os.path.join(path,"{c}.csv".format(c=name_of_flat_input_file)), index=False)